In [1]:
from math import ceil
import copy
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
import numpy.linalg as nla

from sim_lib.simulation import run_simulation
from sim_lib.graph_create import kleinberg_grid, reduce_providers_simplest, powerlaw_dist_time
from sim_lib.sim_strategies import EvenAlloc, MWU
import sim_lib.util as util

init_notebook_mode(connected=True)

In [2]:
# Constants
num_vertices = 10
regularity = 4

# MWU parameters
lrate = 0.5

In [3]:
def create_kg(n, k, p, r):
    return kleinberg_grid(n, n, r, p)

In [4]:
def plot_util_comp(given_utils, r, diam):
    fig = go.Figure()
    
    social_utils, even_utils = given_utils

    #Add averages trace
    p_vals = [ p for (p, uts) in social_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (MWU)', marker=dict(color='Blue')))
    
    even_p_vals = [ p for (p, uts) in even_utils ]
    even_ut_vals = [ np.average(uts) for (p, uts) in even_utils ]
    fig.add_trace(go.Scatter(x=even_p_vals,
        y=even_ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (even alloc)',
        marker=dict(color='Orange')))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in social_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))

    even_indiv_p_vals = []
    even_indiv_ut_vals = []
    for (p, uts) in even_utils:
        even_indiv_ut_vals.extend(uts)
        even_indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=even_indiv_p_vals,
        y=even_indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='rgba(244,204,102,1)'),
        name='social welfare per p (even alloc)'))


    fig.layout.update(showlegend=False)
    title_fmt = "Average Social Welfare (Normalized) vs p (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='avg social welfare (MWU)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [5]:
def plot_dist_err(derr_means, r, diam):
    fig = go.Figure()
    
    dist_names = ['MWU', 'Actual to Hops', 'Closest to Hops', 'Actual to Closest']
    lcolors = ['Blue', 'darkorange', 'darkgreen', 'darkred']
    mcolors = ['LightSkyBlue', 'orange', 'green', 'red']
    
    for idx, derr_vals in enumerate(derr_means):
        name = dist_names[idx]
        lcol = lcolors[idx]
        mcol = mcolors[idx]
        
        #Add averages trace
        p_vals = [ p for (p, derr) in derr_vals ]
        den_avgs = [ np.average(derr) for (p, derr) in derr_vals ]
        fig.add_trace(go.Scatter(x=p_vals,
            y=den_avgs, mode='lines+markers', hoverinfo='skip',
            name=f'avg {name} distance rel error', marker=dict(color=lcol)))

        #Add individual points trace
        indiv_p_vals = []
        indiv_den_vals = []
        for (p, derr) in derr_vals:
            indiv_den_vals.extend(derr)
            indiv_p_vals.extend([p] * len(derr))
        fig.add_trace(go.Scatter(x=indiv_p_vals,
            y=indiv_den_vals, mode='markers', hoverinfo='skip', opacity=0.7,
            marker=dict(color=mcol), name=f'{name} distance rel error per p'))

    fig.layout.update(showlegend=False)
    title_fmt = "Decentralized Search Distance Relative Error vs p (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title=f'{name} Dist Avg Rel Error'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [6]:
def comp_strat(num_vertices, regularity, graph_func, strat, strat_params, plaw_resources=False):
    for r in [ 2 ** k for k in range(1, 8) ][::-1]:
        social_utils = []
        even_utils = []
        
        mwu_dist_err = []
        act_hop_dist_err = []
        opt_hop_dist_err = []
        found_final_dist_err = []
        
        for p in np.linspace(1, 0, 10, endpoint=False):
            utils = []
            eutils = []
            
            mwu_err_means = []
            act_hop_err_means = []
            opt_hop_err_means = []
            found_final_err_means = []
            
            num_iter = 10
            for i in range(num_iter):
                G = graph_func(num_vertices, regularity, p, r)

                graph_diam = util.calc_diameter(G)

                reduce_providers_simplest(G)
                
                #Get opt to find shortest paths later on
                opt_seeds = util.opt_vertices(G)
                initial_utils = set([ v.utility for v in G.vertices ])
                initial_seeds = { ut : [ v for v in G.vertices if v == ut ] 
                                 for ut in initial_utils }
                
                if plaw_resources:
                    powerlaw_dist_time(G, 2)
                    
                #Make copy of graph to run even alloc strategy on
                G_even = copy.deepcopy(G)

                #Initialize strategy
                sim_strat = strat(**strat_params)
                sim_strat.initialize_model(G)

                sim_g, sim_utils, sim_ut_map = run_simulation(G, sim_strat, True)
                
                #Initialize even alloc strategy
                even_strat = EvenAlloc()
                even_strat.initialize_model(G_even)
                
                sim_g_even, sim_utils_even = run_simulation(G_even, even_strat)

                #Get global social welfare at end of simulation normalized by size
                utils.append(sum(sim_utils[-1]) / len(G.vertices))
                eutils.append(sum(sim_utils_even[-1]) / len(G_even.vertices))
                
                #Check decentralized search success
                unif_weight = defaultdict(lambda : defaultdict(lambda : 1))
                
                #Get inverse normalized weights
                mwu_weight = {}
                for vtx in G.vertices:
                    mwu_weight[vtx] = {}
                    tot_weight = sum(sim_strat.vtx_weights[vtx].values())
                    for nbor in vtx.nbors:
                        mwu_weight[vtx][nbor] = 1 - (sim_strat.vtx_weights[vtx][nbor] / tot_weight)
                
                true_apsp, ta_nxt = util.weighted_apsp(G, unif_weight)
                mwu_apsp, ma_nxt = util.weighted_apsp(G, mwu_weight)
                mwu_el_apsp = util.fw_edge_len(G, ma_nxt)
                
                #Get error for closest starting opt vertices
                opt_dist_err = []
                hop_dist_err_act = []
                hop_dist_err_opt = []
                found_opt_dist_err = []
                for vtx in G.vertices:
                    
                    def rel_error(x, y):
                        if y == 0:
                            assert x == y, (x, y)
                            return 0
                        return np.abs(x - y) / y
                    
                    # Get relative error of true distance to closest opt vtx and MWU weighted distance
                    if vtx not in opt_seeds:
                        true_opt_dist = min([ true_apsp[vtx.vnum][ovtx.vnum] for ovtx in opt_seeds ])
                        mwu_opt_dist = min([ mwu_el_apsp[vtx.vnum][ovtx.vnum] for ovtx in opt_seeds ])
                        opt_re = rel_error(mwu_opt_dist, true_opt_dist)
                        opt_dist_err.append(opt_re)
                        
                    num_hops_final = sim_ut_map[vtx]['iter']
                    final_ut_source = sim_ut_map[vtx]['from']
                    final_ut = sim_ut_map[vtx]['ut']
                    
                    closest_same_ut_dist = num_hops_final
                    closest_same_ut_seed = final_ut_source
                    for utx in initial_seeds[final_ut]:
                        if num_hops_final == 0:
                            break
                        if true_apsp[vtx.vnum][utx.vnum] < closest_same_ut_dist:
                            closest_same_ut_dist = true_apsp[vtx.vnum][utx.vnum]
                            closest_same_ut_seed = utx
                    
                    # Get relative error of num iters to final vs shortest path to vtx giving final
                    final_vtx_dist = true_apsp[vtx.vnum][final_ut_source.vnum]
                    iter_sp_re = rel_error(num_hops_final, final_vtx_dist)
                    hop_dist_err_act.append(iter_sp_re)
                    
                    # Get relative error of num iters to final vs shortest path to closest vtx giving final
                    iter_sp_ut_re = rel_error(num_hops_final, closest_same_ut_dist)
                    hop_dist_err_opt.append(iter_sp_ut_re)
                    
                    # Get relative error of shortest path to final vs shortest path to closest vtx
                    # giving final
                    iter_final_ut_re = rel_error(true_apsp[vtx.vnum][final_ut_source.vnum],
                                                 closest_same_ut_dist)
                    found_opt_dist_err.append(iter_final_ut_re)
                    
                mwu_mean = np.mean(np.array(opt_dist_err))
                mwu_err_means.append(mwu_mean)
                
                act_hop_mean = np.mean(np.array(hop_dist_err_act))
                act_hop_err_means.append(act_hop_mean)
                
                opt_hop_mean = np.mean(np.array(hop_dist_err_opt))
                opt_hop_err_means.append(opt_hop_mean)
                
                found_final_mean = np.mean(np.array(found_opt_dist_err))
                found_final_err_means.append(found_final_mean)
                
            social_utils.append((p, utils))
            even_utils.append((p, eutils))
            
            mwu_dist_err.append((p, mwu_err_means))
            act_hop_dist_err.append((p, act_hop_err_means))
            opt_hop_dist_err.append((p, opt_hop_err_means))
            found_final_dist_err.append((p, found_final_err_means))

        plot_util_comp((social_utils, even_utils), r, graph_diam)
        
        plot_dist_err((mwu_dist_err, act_hop_dist_err, opt_hop_dist_err, found_final_dist_err),
                      r, graph_diam)

## Evenly distributed resources

In [7]:
comp_strat(num_vertices, regularity, create_rl,
        MWU, {'lrate' : lrate})

## Powerlaw distributed resources

In [8]:
comp_strat(num_vertices, regularity, create_rl,
         MWU, {'lrate' : lrate}, plaw_resources=True)